In [50]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import ZebraLib as zb

def errorplot(ax, data_x, data_y, color, label):
    ax.errorbar((data_x), data_y, xerr=0.057, yerr=0.0057, fmt='o', ecolor=color, markeredgecolor='black', 
        markerfacecolor=color, label=label, markersize=6, capsize=5)
    #ax.plot((data_x), data_y, color=color)


#Import de dados - Experimento
df = pd.read_excel('D:/UNESP/EngMec-UNESP/Lab. Mec Flu I/Relatório_5/dados.xlsx')
df_carr = df.iloc[0:8] # carregamento
df_des = df.iloc[7::]  # Descarregamento
df_des = df_des.drop(index=11)

In [51]:
df_carr

,vol,t,rot
0,1.920,5.91,0.8
1,1.985,4.89,1.0
2,1.730,3.63,1.2
3,1.680,3.15,1.4
4,2.100,3.82,1.6
5,1.650,2.95,1.8
6,2.080,3.52,2.0
7,1.570,2.37,2.2


In [52]:
#Modelo
def func(x, a, b, c):
    return a*x**2 + x*b + c

#Fit do modelo
popt, pcov = curve_fit(func, df_carr.rot, df_carr.vol/df_carr.t*3.6, p0=[1.9, 1, 0])
popt2, pcov = curve_fit(func, df_des.rot, df_des.vol/df_des.t*3.6, p0=[1.9, 1, 0])

In [53]:
popt

array([-0.32946112,  1.74964004,  0.03242617])

In [54]:
%matplotlib qt
A = 3.6 # L/s to m^3/h

new_x = np.linspace(df_carr['rot'].min(), df_carr['rot'].max())
p1 = zb.fit(df_carr['rot'], (df_carr['vol']/df_carr['t'])*A, 2)
p2 = zb.fit(df_des['rot'], (df_des['vol']/df_des['t'])*A, 2)

fig, ax = plt.subplots()
ax.set(xlim=(0.5, 2.5), ylim=(1, 2.5), xlabel=r'Rotâmetro $[m^3/h]$', ylabel=r'Vazão medida $[m^3/h]$')

errorplot(ax, df_carr['rot'], (df_carr['vol']/df_carr['t'])*A, color='red', label='Carregamento')
errorplot(ax, df_des['rot'], (df_des['vol']/df_des['t'])*A, color='blue', label='Descarregamento')

ax.plot(new_x, func(new_x, *popt), label='Carregamento', ls='--', color='red',)
ax.plot(new_x, func(new_x, *popt2), label='Descarregamento', ls='--', color='blue')

ax.grid(ls='dotted')
ax.legend()

fig.savefig('calib.pdf', bbox_inches='tight')

In [55]:
print(f'{np.poly1d(p1, variable="x")}')
print(f'{np.poly1d(p2, variable="x")}')

         2
-0.3295 x + 1.75 x + 0.03243
         2
-0.5344 x + 2.427 x - 0.3928
